<h1> Lyrics Collector </h1>

This Jupyter notebook can be used to collect lyrics from the Genius API by using the LyricsGenius wrapper.
This notebook has two prerequisites: 

- You need a predefined dataset with Song Title and Artists. (It is based on an export from Exportify)
- You need to register for the Genius API and require your own API key.

In [6]:
# importing libraries
import lyricsgenius as genius
import csv
import json
import pandas as pd
from datetime import datetime
import re 

In [7]:
# set up API , use your own API key
api = genius.Genius("API-KEY", sleep_time=0.01, verbose=False, remove_section_headers= True)

In [10]:
# open the quarantine playlist, download this using exportify
df = pd.read_csv('CSV-DATASET') 

# renaming the columns
df.rename(columns={'Track Name': 'Song Title', 'Artist Name(s)': 'Artist'}, inplace=True)

# select only the bare minimum to prevent larger datasets than needed
df = df[['Artist', 'Song Title']]

df.head(10)

,Artist,Song Title
0,Redman,Tonight's Da Night
1,EPMD,Da Joint
2,"Warren G,O.G.L.B.",This D.J.
3,Erick Sermon,Stay Real
4,MC Serch,Back To The Grill
5,Def Squad,Check N' Me Out
6,Public Enemy,Tie Goes To The Runner
7,Beastie Boys,The New Style
8,Method Man,Bring The Pain
9,Slick Rick,The Ruler's Back


In [11]:
song_data = pd.DataFrame()
start_time = datetime.now()
print("Started at {}".format(start_time))
for i in range(0, len(df)):
    rolling_pct = int((i/len(df))*100)
    song_title = df.iloc[i]['Song Title']
    song_title = re.sub(" and ", " & ", song_title)
    artist_name = df.iloc[i]['Artist']
    artist_name = re.sub(" and ", " & ", artist_name)

    try:
        song = api.search_song(song_title, artist=artist_name)
        #print(song)
        song_lyrics = re.sub("\n", " ", song.lyrics) #Remove newline breaks, we won't need them.

    except:
        song_lyrics = "null"
    
    row = {
        "Song Title": df.iloc[i]['Song Title'],
        "Artist": df.iloc[i]['Artist'],
        "Lyrics": song_lyrics
    }
    song_data = song_data.append(row, ignore_index=True)
end_time = datetime.now()
print("\nCompleted at {}".format(start_time))
print("Total time to collect: {}".format(end_time - start_time))

song_data.head()

Started at 2020-06-11 22:51:58.051259
Timeout raised and caught:
HTTPSConnectionPool(host='api.genius.com', port=443): Read timed out. (read timeout=5)
Timeout raised and caught:
HTTPSConnectionPool(host='api.genius.com', port=443): Read timed out. (read timeout=5)
Timeout raised and caught:
HTTPSConnectionPool(host='api.genius.com', port=443): Read timed out. (read timeout=5)
Timeout raised and caught:
HTTPSConnectionPool(host='api.genius.com', port=443): Read timed out. (read timeout=5)
Timeout raised and caught:
HTTPSConnectionPool(host='api.genius.com', port=443): Read timed out. (read timeout=5)
Timeout raised and caught:
HTTPSConnectionPool(host='api.genius.com', port=443): Read timed out. (read timeout=5)
Timeout raised and caught:
HTTPSConnectionPool(host='api.genius.com', port=443): Read timed out. (read timeout=5)

Completed at 2020-06-11 22:51:58.051259
Total time to collect: 1:13:01.583006


,Artist,Lyrics,Song Title
0,Redman,"Mic check, I can get smooth to any groove Rela...",Tonight's Da Night
1,EPMD,I make a million...bucks Every six months and ...,Da Joint
2,"Warren G,O.G.L.B.",It's kind of easy when you listen to the G'd-u...,This D.J.
3,Erick Sermon,"It's on breaker 1, 2 mike checkin' Flexin no h...",Stay Real
4,MC Serch,"Back to the grill again, the grill again (Repe...",Back To The Grill


In [12]:
# save to .csv and .json
song_data.to_csv("hiphop.csv")
song_data.to_json("hiphop.json", orient='records')

In [13]:
# checking the .csv and seeing how many songs it could not find
hiphop = pd.read_csv('hiphop.csv', index_col=0)
display(hiphop)

,Artist,Lyrics,Song Title
0,Redman,"Mic check, I can get smooth to any groove Rela...",Tonight's Da Night
1,EPMD,I make a million...bucks Every six months and ...,Da Joint
2,"Warren G,O.G.L.B.",It's kind of easy when you listen to the G'd-u...,This D.J.
3,Erick Sermon,"It's on breaker 1, 2 mike checkin' Flexin no h...",Stay Real
4,MC Serch,"Back to the grill again, the grill again (Repe...",Back To The Grill
...,...,...,...
1056,"CHLOBOCOP,Lee Scott",Lyrics for this song ha...,BELL ME BACK
1057,Piers James,As we there are where are we are we there from...,Garden of Eden
1058,"Yeti,Mase","Yo, yo, yo, yo, yo, yo Whoo, whoo, whoo Yeah, ...",Be Mindful
1061,CP,I see u niggas is nervous New D - E leave yo a...,Need Sum
